## Introduction
-----------------------------

In this notebook I'm using [Pantheon](https://github.com/Canisback/pantheon) to define various other functions for using the Riot API. The reason for using Pantheon is to make use of it's rate limiting system, since as a standalone account I can only make so many calls to the API without getting blacklisted.

In [14]:
import json, asyncio, nest_asyncio, datetime
from pantheon import pantheon
nest_asyncio.apply()

For this next cell, I'm going to hide my API key, even though it is reset every 24 hours.

In [15]:
server = "na1"
api_key = "RGAPI-XXXXX"

These are some functions provided in [Pantheon](https://github.com/Canisback/pantheon) that I will use to get any account's summoner ID and recent match history.

In [32]:
def requestsLog(url, status, headers):
    print(url)
    print(status)
    print(headers)

panth = pantheon.Pantheon(server, api_key, errorHandling=True, requestsLoggingFunction=requestsLog, debug=True)

async def getSummonerId(name):
    try:
        data = await panth.getSummonerByName(name)
        return (data['id'],data['accountId'])
    except Exception as e:
        print(e)


async def getRecentMatchlist(accountId):
    try:
        data = await panth.getMatchlist(accountId, params={"endIndex":20})
        return data
    except Exception as e:
        print(e)

async def getRecentMatches(accountId):
    try:
        matchlist = await getRecentMatchlist(accountId)
        tasks = [panth.getMatch(match['gameId']) for match in matchlist['matches']]
        return await asyncio.gather(*tasks)
    except Exception as e:
        print(e)
        

You can choose whatever summoner name you want to search for, I just used my own. 

In [31]:
name = "Jooku"

#call function to set the summonerId and accountId for the name that I chose
loop = asyncio.get_event_loop()  
(summonerId, accountId) = loop.run_until_complete(getSummonerId(name))


https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/Jooku
200
<CIMultiDictProxy('Date': 'Wed, 08 Sep 2021 01:54:56 GMT', 'Content-Type': 'application/json;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'X-App-Rate-Limit': '20:1,100:120', 'X-App-Rate-Limit-Count': '1:1,4:120', 'X-Method-Rate-Limit': '2000:60', 'X-Method-Rate-Limit-Count': '4:60', 'X-Riot-Edge-Trace-Id': '799fb89c-6979-43c1-96e8-93374708e78e', 'Content-Encoding': 'gzip', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': 'GET, PUT, DELETE, POST, OPTIONS', 'Access-Control-Allow-Headers': 'DNT,User-Agent,X-Requested-With,If-Modified-Since,Cache-Control,Content-Type,Range', 'Access-Control-Expose-Headers': 'Content-Length,Content-Range')>
